In [95]:
import geopandas as gpd
import pandas as pd
import shapely
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [136]:
demographic = pd.read_excel('pumadata.xlsx')

In [137]:
demographic.head(3)

,GeoType,GeogName,GeoID,Borough,Pop_1E,Pop_1M,Pop_1C,Pop_1P,Pop_1Z,MaleE,...,AsnOAsnE,AsnOAsnM,AsnOAsnC,AsnOAsnP,AsnOAsnZ,Asn2plE,Asn2plM,Asn2plC,Asn2plP,Asn2plZ
0,PUMA2010,"NYC-Bronx Community District 8--Riverdale, Fie...",3701,Bronx,109810,2134,1.2,100,NaN,51891,...,50,80.0,97.3,0.9,1.4,67,54.0,49.0,1.2,0.9
1,PUMA2010,"NYC-Bronx Community District 12--Wakefield, Wi...",3702,Bronx,144341,2521,1.1,100,NaN,66103,...,11,13.0,71.8,0.3,0.3,11,17.0,93.9,0.3,0.4
2,PUMA2010,"NYC-Bronx Community District 10--Co-op City, P...",3703,Bronx,122417,2742,1.4,100,NaN,56139,...,17,28.0,100.1,0.5,0.8,65,62.0,58.0,1.8,1.7


In [138]:
demographic = demographic[['GeogName','GeoID','Borough','Pop_1E']]
demographic.columns = ['GeogName','GeoID','Borough','Population']

In [140]:
#sum(demographic.Pop_1E)

In [ ]:
# Pop_1 : popolation
# 

In [141]:
soc = pd.read_excel('soc.xlsx')

In [142]:
# households, collegestudents, non US, Highschool
soc = soc[['GeoID','HH1E','EA_GrdPfDE','FbNotCznE','SE_G9t12E']]
soc.columns = ['GeoID','totalHouseholds','CollegeStudents','Immigrants','Highschoolers']
soc.head()

,GeoID,totalHouseholds,CollegeStudents,Immigrants,Highschoolers
0,3701,41718,14443,16220,4460
1,3702,47817,6696,22155,8672
2,3703,47134,8736,8470,5463
3,3704,43954,8210,21900,7348
4,3705,56633,3357,31498,12585


In [143]:
econ = pd.read_excel('econ.xlsx')

In [144]:
# unemployed, meanIncomehousehold, non US, Highschool
econ = econ[['GeoID','CvLFUEm1E','MnHHIncE']]
econ.columns = ['GeoID','unemployed','avgHouseIncome']
econ.head()

,GeoID,unemployed,avgHouseIncome
0,3701,5242,81929
1,3702,9502,61428
2,3703,5492,72072
3,3704,7863,62896
4,3705,11306,36166


In [145]:
pumaData = demographic.merge(soc,left_on='GeoID', right_on='GeoID', how='inner')
pumaData = pumaData.merge(econ,left_on='GeoID', right_on='GeoID', how='inner')

In [146]:
pumaData.head()

,GeogName,GeoID,Borough,Population,totalHouseholds,CollegeStudents,Immigrants,Highschoolers,unemployed,avgHouseIncome
0,"NYC-Bronx Community District 8--Riverdale, Fie...",3701,Bronx,109810,41718,14443,16220,4460,5242,81929
1,"NYC-Bronx Community District 12--Wakefield, Wi...",3702,Bronx,144341,47817,6696,22155,8672,9502,61428
2,"NYC-Bronx Community District 10--Co-op City, P...",3703,Bronx,122417,47134,8736,8470,5463,5492,72072
3,NYC-Bronx Community District 11--Pelham Parkwa...,3704,Bronx,129501,43954,8210,21900,7348,7863,62896
4,"NYC-Bronx Community District 3 & 6--Belmont, C...",3705,Bronx,171849,56633,3357,31498,12585,11306,36166


In [147]:
pumaData.GeoID = pumaData.GeoID.astype(int)

In [148]:
shapeFilePath= 'puma/geo_export_a271959f-d135-40b7-aca2-b79e414a0ecb.shp'

In [149]:
pumasSHP = gpd.GeoDataFrame.from_file(shapeFilePath)

In [150]:
pumasSHP.crs

{'init': u'epsg:4326'}

In [151]:
pumasSHP = pumasSHP[['puma','geometry']]

In [152]:
pumasSHP['puma'] = pumasSHP.puma.astype(int)

In [153]:
pumaGDF = pumasSHP.merge(pumaData,left_on='puma', right_on='GeoID', how='inner')

In [154]:
pumaGDF.head()

,puma,geometry,GeogName,GeoID,Borough,Population,totalHouseholds,CollegeStudents,Immigrants,Highschoolers,unemployed,avgHouseIncome
0,3701,POLYGON ((-73.89641133483133 40.90450452082026...,"NYC-Bronx Community District 8--Riverdale, Fie...",3701,Bronx,109810,41718,14443,16220,4460,5242,81929
1,3702,"POLYGON ((-73.8442314689986 40.86942035096838,...","NYC-Bronx Community District 12--Wakefield, Wi...",3702,Bronx,144341,47817,6696,22155,8672,9502,61428
2,3703,(POLYGON ((-73.78833349834532 40.8346671297593...,"NYC-Bronx Community District 10--Co-op City, P...",3703,Bronx,122417,47134,8736,8470,5463,5492,72072
3,3704,"POLYGON ((-73.84792614069238 40.8713422330779,...",NYC-Bronx Community District 11--Pelham Parkwa...,3704,Bronx,129501,43954,8210,21900,7348,7863,62896
4,4015,POLYGON ((-73.95374853778468 40.63858633758547...,NYC-Brooklyn Community District 14--Flatbush &...,4015,Brooklyn,162030,57115,15391,28929,8529,6843,69462


In [155]:
nighFile = "combined/bars.shp"

In [156]:
nightGDP = gpd.GeoDataFrame.from_file(nighFile)

In [157]:
nightGDP.crs

{'init': u'epsg:4326'}

In [158]:
len(nightGDP)

2440

In [159]:
NightPuma = gpd.sjoin(pumasSHP, nightGDP, how='inner')

In [160]:
len(NightPuma)

2440

In [161]:
NightPuma.drop(['Borough','geometry','name','Location T','Incident Z','City','index_right'],axis=1,inplace=True)

In [162]:
NightPuma.head()

,puma,num_calls,price,rating,review_cou
0,3701,10,2.0,4.0,222
0,3701,18,2.0,2.5,188
0,3701,56,2.0,2.5,188
0,3701,10,2.0,2.5,188
0,3701,50,NaN,5.0,15


In [163]:
np.mean(NightPuma.price)

1.9345971563981044

In [164]:
NightPuma['price'] = NightPuma.price.fillna(2)

In [165]:
## group by
NP1 = NightPuma.groupby('puma').sum().reset_index()[['puma',"num_calls", "review_cou"]]
NP2 = NightPuma.groupby('puma').mean().reset_index()[['puma',"price", "rating"]]
NP = NP1.merge(NP2,left_on='puma', right_on='puma', how='inner')

In [166]:
len(NP)

55

In [167]:
pumaGDF = pumaGDF.merge(NP,left_on='puma', right_on='puma', how='inner')

In [168]:
partyfile = "yelpEvents/yelpEvents.shp"

In [169]:
partGDF = gpd.GeoDataFrame.from_file(partyfile)
partGDF.crs

{'init': u'epsg:4326'}

In [170]:
partyPuma = gpd.sjoin(pumasSHP, partGDF, how='left')

In [173]:
#partyPuma= partyPuma[['puma','name']]
#partyPuma.columns = ['puma','PartyCount']

In [174]:
PP = partyPuma.groupby('puma').count().reset_index()[['puma',"PartyCount"]]

In [175]:
PP.head(2)

,puma,PartyCount
0,3701,17
1,3702,1


In [176]:
len(PP)

55

In [177]:
pumaGDF = pumaGDF.merge(PP,left_on='puma', right_on='puma', how='inner')

In [178]:
pumaGDF.head()

,puma,geometry,GeogName,GeoID,Borough,Population,totalHouseholds,CollegeStudents,Immigrants,Highschoolers,unemployed,avgHouseIncome,num_calls,review_cou,price,rating,PartyCount
0,3701,POLYGON ((-73.89641133483133 40.90450452082026...,"NYC-Bronx Community District 8--Riverdale, Fie...",3701,Bronx,109810,41718,14443,16220,4460,5242,81929,373,1162,1.578947,3.763158,17
1,3702,"POLYGON ((-73.8442314689986 40.86942035096838,...","NYC-Bronx Community District 12--Wakefield, Wi...",3702,Bronx,144341,47817,6696,22155,8672,9502,61428,435,163,2.166667,2.638889,1
2,3703,(POLYGON ((-73.78833349834532 40.8346671297593...,"NYC-Bronx Community District 10--Co-op City, P...",3703,Bronx,122417,47134,8736,8470,5463,5492,72072,799,2772,1.892857,3.625000,1
3,3704,"POLYGON ((-73.84792614069238 40.8713422330779,...",NYC-Bronx Community District 11--Pelham Parkwa...,3704,Bronx,129501,43954,8210,21900,7348,7863,62896,69,513,2.000000,4.200000,0
4,4015,POLYGON ((-73.95374853778468 40.63858633758547...,NYC-Brooklyn Community District 14--Flatbush &...,4015,Brooklyn,162030,57115,15391,28929,8529,6843,69462,367,3168,2.181818,3.818182,3


In [180]:
pumaGDF.to_file("allData/DataByPuma.shp")

In [179]:
len(pumaGDF)

55